# Distributed Inference and Serving

Common distributed inference strategies according to your hardware resources:
- Single GPU
    - No distributed inference.
- Single-node, multi-GPU
    - tensor parallel: model is too large to fit in one GPU but fit in multiple GPUs within a single node.
        - tensor parallel size = number of GPUs
- Multi-node, multi-GPU
    - tensor parallel + pipeline parallel
        - tensor parallel size = number of GPUs per node
        - pipeline parallel size = number of nodes

## Inference memory calculation

$$ M = \frac{P * 4}{32/Q} * 1.2 (GB) $$

- $M$: GPU memory
- $P$: parameters (in Billions, 7B is 7)
- $4$: 4 bytes for each parameter
- $32$: 1 byte is 8 bits, so 4 bytes is 32 bits
- $Q$: quantization bits (e.g., 16 bits, 8 bits, 4 bits)
- $1.2$: 20% overhead

Memory for loading 70B model at 16 bit precision is

$$ \frac{70 * 4}{32/16} * 1.2 = 168GB $$